In [1]:
# pip install selenium-wire
from seleniumwire import webdriver  # <— note seleniumwire instead of selenium
from selenium.webdriver.common.by import By
import time
import json

TOKEN_URL = "https://swisstennisch.b2clogin.com/swisstennisch.onmicrosoft.com/" \
            "b2c_1a_b2c_1_signup-signin/oauth2/v2.0/token"

driver = webdriver.Chrome()

try:
    # 1) Log in as before:
    driver.get("https://www.mytennis.ch/fr")
    time.sleep(2)
    driver.find_element(By.ID, "UserId").send_keys("benoitphilippe")
    driver.find_element(By.ID, "password").send_keys("Tennisman14.")
    driver.find_element(By.ID, "next").click()
    time.sleep(10)

    # 2) Find the token request and print its JSON response
    for req in driver.requests:
        if req.method == "POST" and req.url.startswith(TOKEN_URL):
            # wait until the server has sent a response
            if req.response:
                body = req.response.body.decode("utf-8")
                token_data = json.loads(body)
                print("Status:", req.response.status_code)
                print("JSON response:")
                print(json.dumps(token_data, indent=2))
                break
    else:
        print("No token request found—maybe it hasn't fired yet.")
finally:
    driver.quit()


Status: 200
JSON response:
{
  "id_token": "eyJhbGciOiJSUzI1NiIsImtpZCI6Il96XzBaVG1qVC1vS3NyWkdtLWJ3Q0piUnphcGlWcXBwNWhLUmJwU2ZzZFUiLCJ0eXAiOiJKV1QifQ.eyJleHAiOjE3NDg0MjM4MDksIm5iZiI6MTc0ODQyMDIwOSwidmVyIjoiMS4wIiwiaXNzIjoiaHR0cHM6Ly9zd2lzc3Rlbm5pc2NoLmIyY2xvZ2luLmNvbS84Yjk2NmEyZC02NmNhLTQzMjgtOTAxNS1kYWQxYWFiNmQxNjQvdjIuMC8iLCJzdWIiOiIyZmFjZTNiZi03ZDNmLTQ5MDEtOTM1OC0xNDU2ODc0YzE5OTMiLCJhdWQiOiI1YzhjNmU0ZS1hMDE3LTQ5ZDQtYTljMC1mODlmZTA4NjE0ZjMiLCJub25jZSI6Ijg1MzVkY2QzLWU3MWItNGU3OS1hMzU2LWM0YzI4ZDcyNTIzYiIsImlhdCI6MTc0ODQyMDIwOSwiYXV0aF90aW1lIjoxNzQ4NDIwMjA2LCJleHRlbnNpb25fT3JkaW5hcnlFbWFpbCI6Imxlb2RhZ2FuZmVkZXJlckBob3RtYWlsLmZyIiwiZ2l2ZW5fbmFtZSI6IlBoaWxpcHBlIiwiZmFtaWx5X25hbWUiOiJCZW5vaXQiLCJleHRlbnNpb25fR2VuZGVyIjoibWFsZSIsImV4dGVuc2lvbl9MYW5ndWFnZSI6ImZyIiwiZXh0ZW5zaW9uX1BlcnNvbk5yIjoiMTk4NjA2NTYiLCJleHRlbnNpb25fUm9sZXMiOiIiLCJleHRlbnNpb25fRmF2b3JpdGVzIjoiIiwiZXh0ZW5zaW9uX0Fzc29jaWF0ZVR5cGVJRCI6IiIsInRmcCI6IkIyQ18xQV9CMkNfMV9zaWdudXAtc2lnbmluIn0.rFvAXEqde5pQrFuz-VVJrICClvAKR39Onru8U

# get all players

In [2]:

import requests

url = "https://hasura.swisstennis.ch/v1/graphql"
headers = {"authorization": f"Bearer {token_data['id_token']}", "content-type": "application/json"}


In [3]:
payload = {
  "query": "query PlayerList($offset: Int, $limit: Int, $id: Int, $order_by: [lizenz_nehmer_order_by!], $where: lizenz_nehmer_bool_exp) {\n  lizenz_nehmer(\n    offset: $offset\n    limit: $limit\n    where: $where\n    order_by: $order_by\n  ) {\n    licenceNumber\n    classification\n    classificationValue\n    competitionValue\n    ageCategoryRedundant\n    currentLicenceStatusId\n    ranking\n    person {\n      id\n      firstname\n      lastname\n    }\n  }\n  user_profile_frienship(where: {personId: {_eq: $id}}) {\n    playerId\n  }\n}",
  "variables": {
    "id": 19860656,
    "offset": 0,
    "limit": 10000,
    "where": {
      "kontingent": {
        "_eq": 1
      },
      "currentLicenceStatusId": {
        "_lt": 3
      },
      "person": {
        "gender": {
          "_eq": 1
        }
      }
    },
    "order_by": [
      {
        "classificationValue": "desc"
      },
      {
        "ranking": "asc"
      }
    ]
  },
  "operationName": "PlayerList"
}
response = requests.post(url, json=payload, headers=headers)
print(response.status_code)


200


In [4]:
data = response.json()
data

{'data': {'lizenz_nehmer': [{'licenceNumber': '936.85.190.0',
    'classification': 'N1',
    'classificationValue': 16.39,
    'competitionValue': 15.299,
    'ageCategoryRedundant': '40+',
    'currentLicenceStatusId': 2,
    'ranking': 1,
    'person': {'id': 70318, 'firstname': 'Stan', 'lastname': 'Wawrinka'}},
   {'licenceNumber': '497.96.286.0',
    'classification': 'N1',
    'classificationValue': 15.945,
    'competitionValue': 14.816,
    'ageCategoryRedundant': 'A',
    'currentLicenceStatusId': 1,
    'ranking': 2,
    'person': {'id': 126520,
     'firstname': 'Marc-Andrea',
     'lastname': 'Hüsler'}},
   {'licenceNumber': '574.03.143.0',
    'classification': 'N1',
    'classificationValue': 15.787,
    'competitionValue': 14.604,
    'ageCategoryRedundant': 'A',
    'currentLicenceStatusId': 1,
    'ranking': 3,
    'person': {'id': 172242, 'firstname': 'Jérôme', 'lastname': 'Kym'}},
   {'licenceNumber': '748.94.186.0',
    'classification': 'N1',
    'classificationVal

In [5]:
import pandas as pd


df = pd.json_normalize(data['data']['lizenz_nehmer'])

# Create 'name' column by combining first and last names
df['name'] = df['person.firstname'] + ' ' + df['person.lastname']
df.columns=[c.split("person.")[-1] for c in df.columns]
df.to_parquet("data/all_players.parquet")


# each player

In [6]:
import pandas as pd
all_players = pd.read_parquet("data/all_players.parquet")
all_players

,licenceNumber,classification,classificationValue,competitionValue,ageCategoryRedundant,currentLicenceStatusId,ranking,id,firstname,lastname,name
0,936.85.190.0,N1,16.390,15.299,40+,2,1,70318,Stan,Wawrinka,Stan Wawrinka
1,497.96.286.0,N1,15.945,14.816,A,1,2,126520,Marc-Andrea,Hüsler,Marc-Andrea Hüsler
2,574.03.143.0,N1,15.787,14.604,A,1,3,172242,Jérôme,Kym,Jérôme Kym
3,748.94.186.0,N1,15.735,14.644,A,1,4,122182,Alexander,Ritschard,Alexander Ritschard
4,868.02.347.0,N1,15.539,14.558,A,1,5,158613,Dominic,Stricker,Dominic Stricker
...,...,...,...,...,...,...,...,...,...,...,...
9995,433.87.491.0,R6,3.531,3.018,35+,1,9815,19846575,Sven,Hagedorn,Sven Hagedorn
9996,750.76.420.0,R6,3.531,3.003,45+,1,9816,19821719,Mark,Robinson,Mark Robinson
9997,232.84.151.1,R6,3.530,3.298,40+,1,9817,19754360,Andreas,Bühlmann,Andreas Bühlmann
9998,911.99.481.0,R6,3.530,3.286,A,1,9818,164860,Jonas,Vetsch,Jonas Vetsch


In [14]:
for player in all_players.itertuples():
    print(player)
    break

Pandas(Index=0, licenceNumber='936.85.190.0', classification='N1', classificationValue=16.39, competitionValue=15.299, ageCategoryRedundant='40+', currentLicenceStatusId=2, ranking=1, id=70318, firstname='Stan', lastname='Wawrinka', name='Stan Wawrinka')


In [8]:
from tqdm import tqdm
import requests
i=0
for player in tqdm(all_players.itertuples(), total=len(all_players)):
    i+=1
    # if i < 1370:
    #     continue
    payload = {
        "query":
        "query SingleResults($where: AllSingleResults_bool_exp, $orderBy: [AllSingleResults_order_by!], $limit: Int = 300) {\n  results: AllSingleResults(where: $where, order_by: $orderBy, limit: $limit) {\n    ...singleResult\n  }\n}\n\nfragment singleResult on AllSingleResults {\n  date\n  tournamentName\n  adversaryPersonId\n  adversaryFirstname\n  adversaryLastname\n  playerSet1WonGames\n  adversarySet1WonGames\n  playerSet2WonGames\n  adversarySet2WonGames\n  playerSet3WonGames\n  adversarySet3WonGames\n  playerSet4WonGames\n  adversarySet4WonGames\n  playerSet5WonGames\n  adversarySet5WonGames\n  playerWinnerCode\n  Asr_AdversaryValue_f\n  adversary {\n    classification\n    classificationValue\n    competitionValue4L\n    ranking\n  }\n  ATPWTARank\n  Asr_Id_Item_l\n  round: Asr_Round_i\n  encounterId: Asr_Id_Item_l\n  matchNotConsidered\n  event {\n    id\n    tournamentId\n    eventModeId\n  }\n  source\n  rank: Asr_RankConvert_s\n}",
        "variables": {
            "where": {
                "playerPersonId": {
                    "_eq": player.id
                },
                "playerWinnerCode": {
                    "_neq": "D"
                },
                # "source": {
                #     "_in": [110, 111, 120, 121, 150, 410, 411, 420, 421, 450, 500, 501, 600, 601, 660, 610, 650]
                # },
                "rankSeasonRange": {
                    "seasonPointer": {
                        "_in": [0, 1]
                    }
                }
            },
            "orderBy": {
                "date": "desc"
            }
        },
        "operationName": "SingleResults"
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data=response.json()["data"]["results"]
    df = pd.json_normalize(data)
    if len(df) == 0:
        df.to_parquet(f"data/singles/{player.name.replace(' ','-')}_competitionValue={player.competitionValue}.parquet")
        continue
    wanted = [
        "date",
        "adversaryFirstname",
        "adversaryLastname",
        "playerWinnerCode",
        "adversary.classification",
        "rank",
        "adversary.competitionValue4L",
        "Asr_AdversaryValue_f",
    ]
    existing = [col for col in wanted if col in df.columns]
    df = df[existing]
    
    df["date"] = pd.to_datetime(df["date"])
    df=df.set_index("date").sort_index(ascending=False)
    if "adversary.classification" in df.columns:
        df["adversary.classification"] = df["adversary.classification"].fillna(df["rank"])
    if "adversary.competitionValue4L" not in df.columns:
        df["adversary.competitionValue4L"] = df["Asr_AdversaryValue_f"]
    else:
        df["adversary.competitionValue4L"] = df["adversary.competitionValue4L"].fillna(df["Asr_AdversaryValue_f"])
    
    wanted=[
        "adversaryFirstname",
        "adversaryLastname",
        "playerWinnerCode",
        "adversary.classification",
        "adversary.competitionValue4L",
    ]
    existing = [col for col in wanted if col in df.columns]
    df = df[existing]
    df = df[df["adversary.competitionValue4L"] != 0]
    
    df.to_parquet(f"data/singles/{player.name.replace(' ','-')}_competitionValue={player.competitionValue}.parquet")
    

 30%|███       | 3040/10000 [12:47<29:17,  3.96it/s] 


KeyboardInterrupt: 

In [165]:
len(df)

0

In [111]:
data=response.json()["data"]["results"]
df = pd.json_normalize(data)
df = df[[
    "date",
    "adversaryFirstname",
    "adversaryLastname",
    "playerWinnerCode",
    "adversary.classification",
    "rank",
    "adversary.competitionValue4L",
    "Asr_AdversaryValue_f",
]]
df["date"] = pd.to_datetime(df["date"])
df=df.set_index("date").sort_index(ascending=False)
df["adversary.classification"] = df["adversary.classification"].fillna(df["rank"])
df["adversary.competitionValue4L"] = df["adversary.competitionValue4L"].fillna(df["Asr_AdversaryValue_f"])
df = df[[
    "adversaryFirstname",
    "adversaryLastname",
    "playerWinnerCode",
    "adversary.classification",
    "adversary.competitionValue4L",
]]
df


,adversaryFirstname,adversaryLastname,playerWinnerCode,adversary.classification,adversary.competitionValue4L
date,,,,,
2025-05-24 12:00:00,Alejo,Prado,N,N3,11.728000
2025-05-10 12:00:00,Noël,Kunz,S,R2,8.220000
2025-05-03 12:00:00,Björn,Schilter,N,R2,8.069000
2025-04-26 00:00:00,William,Kjellberg,N,R1,9.207924
2025-04-26 00:00:00,Paolo,Iallonardo,S,R1,9.098000
2025-04-26 00:00:00,Daniele,Chiappini,S,R2,8.062016
2025-04-19 00:00:00,Benjamin,Freyholdt,S,N4,9.764000
2025-04-19 00:00:00,Felix,Seres,N,N4,11.061000
2025-04-19 00:00:00,Petr,Minakhin,S,R1,9.207924
